# Financial Data Analysis

**Goal:** Get comfortable with data manipulation and visualization.\
**Skills Learned:** Pandas, Matplotlib, Seaborn, data cleaning.\
**Ideas:**
- Import historical stock prices (using yfinance) and compute returns, moving averages, and volatility.
- Compare multiple stocks’ performance over time.
- Calculate correlation between different assets.

## Import historical stock prices

In [ ]:
import yfinance as yf
